In [18]:
import numpy as np
import pandas as pd
import world_bank_data as wb

# used for suppressing warning
import warnings 
warnings.filterwarnings('ignore') # suppress the warnings

In [19]:
# https://www.fao.org/faostat/en/#data/QCL
df = pd.read_csv('Production_Crops_Livestock_E_All_Data_NOFLAG.csv', encoding= 'unicode_escape')

In [20]:
def remove0orNAN(df, colList=None):

    newdf = df
    
    if colList != None:
        for x in colList:
            newdf[x] = newdf[x].astype(str)
        
    # create a new column indicating if all numeric columns are NaN
    # I summed the columns in the row, any with 0 or NaN can be removed.
    newdf["Remove0orNaN"] = (df.sum(axis = 1, skipna = True, numeric_only = True)).replace(0, np.nan) 

    # drop rows with NaN in all numeric columns and drop check column
    newdf = newdf.dropna(subset = ["Remove0orNaN"]).drop(columns = ["Remove0orNaN"])
    
    return newdf

In [21]:
def splitdf(title, element, df):
    
    newdf = pd.DataFrame()
    newdf = df[df.Element == element]
    newdf = pd.melt(newdf, 
                    id_vars = ["Area Code", "Area", "Item Code", "Item", "Element Code", "Element", "Unit"], 
                    var_name = "Year", 
                    value_name = title)
    
    try:
        
        newdf[title] = np.where(newdf["Unit"] == "1000 Head",
                                   newdf[title] * 1000, newdf[title])
        
        newdf["Unit"] = newdf["Unit"].str.replace("1000 ","")
        
        newdf.drop(columns = ["Element", "Element Code", "Unit"], inplace = True)
        
        newdf["Year"] = newdf["Year"].str.replace("Y","").astype(int)
        
        return newdf
    
    except:
        
        newdf.drop(columns = ["Element", "Element Code", "Unit"], inplace = True)
        
        newdf["Year"] = newdf["Year"].str.replace("Y","").astype(int)
        
        return newdf

In [22]:
def filterdf(df, column, pick):
    
    find = df[column].apply(lambda x: any(item for item in pick if item in x))
    newdf = df[find]
    
    return newdf

In [23]:
def groupReplace(df, column, replaceList):
    
    newdf = df
    
    for x in replaceList:
        j, k = x
        newdf[column] = newdf[column].str.replace(j, k)
    
    return newdf

In [24]:
def displayInfo(df):
    
    # display the amount of missing values
    print("Sum of all missing values: ", 
          df.isnull().sum().sum(), "\n")
    print(df.info())
    return df
    

In [25]:
colList = ['Area Code', 'Item Code', 'Element Code']

df = remove0orNAN(df, colList)

In [26]:
dairy = ['milk', 'Milk', 'Cream', 'Butter', 'Cheese', 'Yoghurt']

dfDairy = filterdf(df, 'Item', dairy)

dairy_prod = splitdf("Prod_Tonnes", "Production", dfDairy)

dairy_yield = splitdf("Yield_hg_An", "Yield", dfDairy)
dairy_yield = dairy_yield[~dairy_yield.Item.str.contains("Milk, Total")]

dairy_animals = splitdf("Animals_Head", "Milk Animals", dfDairy)
dairy_animals = dairy_animals[~dairy_animals.Item.str.contains("Milk, Total")]

displayInfo(dairy_prod)

Sum of all missing values:  30223 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160620 entries, 0 to 160619
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Area Code    160620 non-null  object 
 1   Area         160620 non-null  object 
 2   Item Code    160620 non-null  object 
 3   Item         160620 non-null  object 
 4   Year         160620 non-null  int64  
 5   Prod_Tonnes  130397 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 7.4+ MB
None


,Area Code,Area,Item Code,Item,Year,Prod_Tonnes
0,2,Afghanistan,983,"Butter and ghee, sheep milk",1961,4104.0
1,2,Afghanistan,886,"Butter, cow milk",1961,7000.0
2,2,Afghanistan,1021,"Cheese, goat milk",1961,3150.0
3,2,Afghanistan,984,"Cheese, sheep milk",1961,10944.0
4,2,Afghanistan,888,"Milk, skimmed cow",1961,131600.0
...,...,...,...,...,...,...
160615,5817,Net Food Importing Developing Countries,1811,Butter and Ghee,2020,NaN
160616,5817,Net Food Importing Developing Countries,1745,Cheese (All Kinds),2020,NaN
160617,5817,Net Food Importing Developing Countries,1816,Evaporat&Condensed Milk,2020,NaN
160618,5817,Net Food Importing Developing Countries,1780,"Milk, Total",2020,123486104.0


In [27]:
# merge the DFs on country and year. This works the equivalent of a vlookup in excel
milk_df = dairy_animals.merge(dairy_yield, 
                              how = 'left', 
                              on = ('Area Code', 'Area', 'Item Code','Item', 'Year'))
milk_df = milk_df.merge(dairy_prod, 
                        how = 'left', 
                        on = ('Area Code', 'Area', 'Item Code','Item', 'Year'))

replace = [["Milk, whole fresh ",""], ["cow","Cattle"]]
milk_df = groupReplace(milk_df, 'Item', replace)
milk_df["Item"] = milk_df["Item"].str.capitalize()

displayInfo(milk_df)

Sum of all missing values:  16058 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35040 entries, 0 to 35039
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Area Code     35040 non-null  object 
 1   Area          35040 non-null  object 
 2   Item Code     35040 non-null  object 
 3   Item          35040 non-null  object 
 4   Year          35040 non-null  int64  
 5   Animals_Head  29658 non-null  float64
 6   Yield_hg_An   29510 non-null  float64
 7   Prod_Tonnes   29894 non-null  float64
dtypes: float64(3), int64(1), object(4)
memory usage: 2.4+ MB
None


,Area Code,Area,Item Code,Item,Year,Animals_Head,Yield_hg_An,Prod_Tonnes
0,2,Afghanistan,1130,Camel,1961,25000.0,3000.0,7500.0
1,2,Afghanistan,882,Cattle,1961,700000.0,5000.0,350000.0
2,2,Afghanistan,1020,Goat,1961,834000.0,420.0,35000.0
3,2,Afghanistan,982,Sheep,1961,6080000.0,300.0,182400.0
4,3,Albania,951,Buffalo,1961,1150.0,4348.0,500.0
...,...,...,...,...,...,...,...,...
35035,5817,Net Food Importing Developing Countries,951,Buffalo,2020,19457898.0,20972.0,40807828.0
35036,5817,Net Food Importing Developing Countries,1130,Camel,2020,7463700.0,3877.0,2893502.0
35037,5817,Net Food Importing Developing Countries,882,Cattle,2020,94827100.0,7262.0,68860916.0
35038,5817,Net Food Importing Developing Countries,1020,Goat,2020,135565295.0,631.0,8548336.0


In [28]:
livestock = ['Poultry', 'Beehives', 'Asses', 'Meat', 'Buffaloes', 'Rodent',
             'Camels', 'Cattle', 'Chickens', 'Fat', 'Goats', 'Hides', 'Turkey',
             'Horses', 'Lard', 'Mules', 'Offals', 'Pigs', 'Birds', 'Beehives', 
             'Rabbits', 'Sheep', 'Skins', 'Ducks', 'Geese', 'Camelids']

dfLivestock = filterdf(df, 'Item', livestock)

livestock_head = splitdf("Stock_Head", "Stocks", dfLivestock)

groups = ("Sheep and Goat", "Cattle and Buffaloes", "Poultry", "Beef and Buffalo Meat", "bird", "nes", "game")    

replace = [["Asses","Ass"], ["Chickens","Chicken"], ["Goats","Goat"],
           ["Ducks","Duck"], ["Geese and guinea fowls","Goose and guinea fowl"], 
           ["Rabbits and hares","Rabbit"], ["Camels","Camel"], ["Mules","Mule"], 
           ["Camelids, other","Other camelids"], ["Rodents, other","Other rodents"], 
           ["Turkeys","Turkey"], ["Pigs","Pig"], ["Horses","Horse"], ["Buffaloes","Buffalo"]]

livestock_head = groupReplace(livestock_head, 'Item', replace)

livestock_slaughtered = splitdf("Slaughter_Head", "Producing Animals/Slaughtered", dfLivestock)

for x in groups:
    livestock_slaughtered = livestock_slaughtered[~livestock_slaughtered.Item.str.contains(x)]
    livestock_head = livestock_head[~livestock_head.Item.str.contains(x)]
    
# rename columns
livestock_head = livestock_head.rename(columns={"Item":"Animal"})

In [29]:
remove = ("Fat", "Hides", "fresh", "Meat", 
          "edible", "Skins", "Offals", ", ")

livestock_slaughtered["Animal"] = livestock_slaughtered["Item"]

for x in remove:
    livestock_slaughtered["Animal"] = livestock_slaughtered["Animal"].str.replace(x,"")
    
replace = [["Horses","Horse"], ["Pigs","Pig"], ["Buffaloes","Buffalo"],
           ["Camels","Camel"], ["Goats","Goat"]]

livestock_slaughtered["Animal"] = livestock_slaughtered["Animal"].str.capitalize()
livestock_slaughtered = groupReplace(livestock_slaughtered, 'Animal', replace)

livestock_slaughtered = livestock_slaughtered[["Area Code", "Area", "Animal", "Year", "Slaughter_Head"]]
livestock_slaughtered = livestock_slaughtered.dropna(how = 'any').drop_duplicates()

In [30]:
# merge the DFs on country and year. This works the equivalent of a vlookup in excel
livestock_df = livestock_head.merge(livestock_slaughtered, how = 'left', on = ('Area Code','Area', 'Animal', 'Year'))

livestock_df_na = livestock_df[livestock_df["Stock_Head"].isna()]
livestock_df_na[livestock_df_na["Slaughter_Head"] != 0]

#livestock_df['Slaughter_Head'].fillna(0, inplace=True)
#livestock_df['Stock_Head'].fillna(0, inplace=True)

livestock_df = remove0orNAN(livestock_df, ['Year'])

displayInfo(livestock_df)

Sum of all missing values:  43718 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 140168 entries, 0 to 159717
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Area Code       140168 non-null  object 
 1   Area            140168 non-null  object 
 2   Item Code       140168 non-null  object 
 3   Animal          140168 non-null  object 
 4   Year            140168 non-null  object 
 5   Stock_Head      139804 non-null  float64
 6   Slaughter_Head  96814 non-null   float64
dtypes: float64(2), object(5)
memory usage: 8.6+ MB
None


,Area Code,Area,Item Code,Animal,Year,Stock_Head,Slaughter_Head
0,2,Afghanistan,1107,Ass,1961,1300000.0,NaN
1,2,Afghanistan,1126,Camel,1961,250000.0,20000.0
2,2,Afghanistan,866,Cattle,1961,2900000.0,360000.0
3,2,Afghanistan,1057,Chicken,1961,4700000.0,7000000.0
4,2,Afghanistan,1016,Goat,1961,4200000.0,940000.0
...,...,...,...,...,...,...,...
159713,5817,Net Food Importing Developing Countries,1140,Rabbit,2020,12872000.0,77755000.0
159714,5817,Net Food Importing Developing Countries,1150,Other rodents,2020,18634000.0,52851000.0
159715,5817,Net Food Importing Developing Countries,976,Sheep,2020,406555086.0,131031821.0
159716,5817,Net Food Importing Developing Countries,1079,Turkey,2020,29376000.0,30789000.0


In [31]:
# get the indicators for everything under the agriculture topic
# topic and source id are from get_topics/get_sources - topic 1 is agriculture
agriculture = wb.get_indicators(topic=1) 
ag_indicator = agriculture.reset_index()
ag_indicator = ag_indicator[["id", "name", "sourceNote", "sourceOrganization"]]
ag_indicator.name.unique()

array(['Agricultural machinery, tractors',
       'Fertilizer consumption (% of fertilizer production)',
       'Fertilizer consumption (kilograms per hectare of arable land)',
       'Agricultural land (sq. km)', 'Agricultural land (% of land area)',
       'Arable land (hectares)', 'Arable land (hectares per person)',
       'Arable land (% of land area)',
       'Land under cereal production (hectares)',
       'Permanent cropland (% of land area)',
       'Rural land area where elevation is below 5 meters (sq. km)',
       'Rural land area where elevation is below 5 meters (% of total land area)',
       'Forest area (sq. km)', 'Forest area (% of land area)',
       'Agricultural irrigated land (% of total agricultural land)',
       'Average precipitation in depth (mm per year)',
       'Land area (sq. km)', 'Rural land area (sq. km)',
       'Agricultural machinery, tractors per 100 sq. km of arable land',
       'Cereal production (metric tons)',
       'Crop production index (2

In [32]:
# create a df with all country locations
countries_loc = wb.get_countries()

# filling in the missing latitudes and longitudes

# https://latitudelongitude.org/gi/gibraltar/
countries_loc['capitalCity'].loc['GIB'] = "Gibraltar";
countries_loc['latitude'].loc['GIB'] = 36.14474;
countries_loc['longitude'].loc['GIB'] = -5.35257;

# https://latitudelongitude.org/tw/taipei/
countries_loc['capitalCity'].loc['TWN'] = "Taipei";
countries_loc['latitude'].loc['TWN'] = 25.04776;
countries_loc['longitude'].loc['TWN'] = 121.53185;

# https://latitudelongitude.org/cw/willemstad/
countries_loc['latitude'].loc['CUW'] = 12.1084;
countries_loc['longitude'].loc['CUW'] = -68.93354;

# https://latitudelongitude.org/je/saint-helier/
countries_loc['capitalCity'].loc['CHI'] = "St. Helier";
countries_loc['latitude'].loc['CHI'] = 49.18804;
countries_loc['longitude'].loc['CHI'] = -2.10491;

# https://latitudelongitude.org/mf/marigot/
countries_loc['latitude'].loc['MAF'] = 18.06667;
countries_loc['longitude'].loc['MAF'] = -63.08333;

# https://latitudelongitude.org/ps/gaza/
countries_loc['capitalCity'].loc['PSE'] = "Gaza";
countries_loc['latitude'].loc['PSE'] = 31.5;
countries_loc['longitude'].loc['PSE'] = 34.46667;

# https://latitudelongitude.org/sx/philipsburg/
countries_loc['latitude'].loc['SXM'] = 18.026;
countries_loc['longitude'].loc['SXM'] = -63.04582;

countries_loc['capitalCity'].loc['HKG'] = "Victoria";
countries_loc['capitalCity'].loc['ISR'] = "Jerusalem";
countries_loc['capitalCity'].loc['MAC'] = "Macau";

# create a df to merge for country codes, latitude and lontitude
temp = countries_loc.reset_index().drop(axis=0, 
                                        columns={"region", "incomeLevel", "capitalCity"})
temp = temp[["name", "id", "iso2Code", "longitude", "latitude"]
           ].rename(columns = {"name":"Country"})

# create a df with countries only for latitude and longitude location
countries_loc = countries_loc.dropna(axis = 0, how ='any'
                                    ).drop(axis=0, 
                                           columns={"adminregion", "lendingType"})

In [33]:
landSqM_df = pd.DataFrame(wb.get_series('AG.LND.AGRI.K2')).reset_index()

# rename columns
landSqM_df = landSqM_df.rename(columns = {"AG.LND.AGRI.K2":"Agr_land_sq_km"}
                               ).drop(axis=0, columns={"Series"})

landPer_df = pd.DataFrame(wb.get_series('AG.LND.AGRI.ZS')).reset_index()

# rename columns
landPer_df = landPer_df.rename(columns = {"AG.LND.AGRI.ZS":"Agr_land_per"}
                               ).drop(axis=0, columns={"Series"})

# merge the DFs on country and year. This works the equivalent of a vlookup in excel
agrLand_df = landSqM_df.merge(landPer_df, how = 'left', on = ('Country', 'Year'))

# merge the country codes and locations on name. This works the equivalent of a vlookup in excel
agrLand_df = agrLand_df.merge(temp, how = 'left', on = ('Country'))

agrLand_country_df = agrLand_df.dropna(subset = ["longitude"]).reset_index()
agrLand_country_df = agrLand_country_df[["id", 
                                         "iso2Code", 
                                         "Country", 
                                         "longitude", 
                                         "latitude", 
                                         "Year", 
                                         "Agr_land_sq_km", 
                                         "Agr_land_per"]]

blank_years = ("2019", "2020", "1960")

for x in blank_years:
    agrLand_country_df = agrLand_country_df[agrLand_country_df["Year"].str.contains(x) == False]

agrLand_country_df["Year"] = agrLand_country_df["Year"].astype(int)    
    
# Saving the frame
agrLand_country_df.to_csv('agrLand_country_df.csv')

displayInfo(agrLand_country_df)

Sum of all missing values:  2882 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12586 entries, 1 to 13234
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              12586 non-null  object 
 1   iso2Code        12586 non-null  object 
 2   Country         12586 non-null  object 
 3   longitude       12586 non-null  float64
 4   latitude        12586 non-null  float64
 5   Year            12586 non-null  int64  
 6   Agr_land_sq_km  11145 non-null  float64
 7   Agr_land_per    11145 non-null  float64
dtypes: float64(4), int64(1), object(3)
memory usage: 885.0+ KB
None


,id,iso2Code,Country,longitude,latitude,Year,Agr_land_sq_km,Agr_land_per
1,AFG,AF,Afghanistan,69.1761,34.5228,1961,377000.0,57.745918
2,AFG,AF,Afghanistan,69.1761,34.5228,1962,377600.0,57.837821
3,AFG,AF,Afghanistan,69.1761,34.5228,1963,378100.0,57.914407
4,AFG,AF,Afghanistan,69.1761,34.5228,1964,378730.0,58.010906
5,AFG,AF,Afghanistan,69.1761,34.5228,1965,378750.0,58.013969
...,...,...,...,...,...,...,...,...
13230,ZWE,ZW,Zimbabwe,31.0672,-17.8312,2014,162000.0,41.876696
13231,ZWE,ZW,Zimbabwe,31.0672,-17.8312,2015,162000.0,41.876696
13232,ZWE,ZW,Zimbabwe,31.0672,-17.8312,2016,162000.0,41.876696
13233,ZWE,ZW,Zimbabwe,31.0672,-17.8312,2017,162000.0,41.876696


In [34]:
df = pd.read_csv("FAOSTAT_livestock_dairy_exports.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'FAOSTAT_livestock_dairy_exports.csv'

In [ ]:
df["Value"] = np.where(df["Unit"].str.contains("1000 ") == True, df["Value"] * 1000, df["Value"])
df["Unit"] = df["Unit"].str.replace("1000 ","")

df.drop(axis=0, columns={"Domain Code", 
                         "Domain", 
                         "Reporter Country Code (FAO)", 
                         "Reporter Countries",
                         "Year Code",
                         "Flag",
                         "Flag Description"}, inplace = True)

exportQ_df = df[df["Element"].str.contains("Export Quantity")==True].drop(axis=0, columns={"Element Code", "Element"})
exportUSD_df = df[df["Element"].str.contains("Export Value")==True].drop(axis=0, columns={"Element Code", "Element"})

In [ ]:
# merge the DFs on country and year. This works the equivalent of a vlookup in excel
export_df = exportQ_df.merge(exportUSD_df, how = 'left', on = ('Partner Country Code (FAO)',
                                                               'Partner Countries', 
                                                               'Item Code', 
                                                               'Item',
                                                               'Year'))

colList = ["Partner Country Code (FAO)", "Item Code", "Year"]

export_df = remove0orNAN(export_df, colList)

displayInfo(export_df)

In [ ]:
export0_df = export_df[export_df["Value_x"] == 0]

In [ ]:
exportnot0_df = export_df[export_df["Value_x"] != 0]
exportnot0_df["Price"] = exportnot0_df["Value_y"] / exportnot0_df["Value_x"]

displayInfo(exportnot0_df)
#exportnot0_df[exportnot0_df["Item"] == "Cattle"]

In [ ]:
# https://www.fao.org/faostat/en/#data/QCL
df = pd.read_csv('Prices_E_All_Data_NOFLAG.csv', encoding= 'unicode_escape')
df = df.drop(columns = {"Months Code", "Months"})

In [ ]:
dairy = ['milk', 'Milk', 'Cream', 'Butter', 'Cheese', 'Yoghurt']
dfDairyPx = filterdf(df, 'Item', dairy)
dfDairyPx = splitdf("Milk_Px_ton", "Producer Price (USD/tonne)", dfDairyPx)
replace = [["Milk, whole fresh ",""], ["cow","Cattle"]]
dfDairyPx = groupReplace(dfDairyPx, 'Item', replace)
dfDairyPx["Item"] = dfDairyPx["Item"].str.capitalize()
dfDairyPx.drop(axis=0, columns={"Area Code", "Item Code"}, inplace = True)

livestockPx = filterdf(df, 'Item', ['Meat live weight, '])
livestockPx = splitdf("Livestock_Px_ton", "Producer Price (USD/tonne)", livestockPx)
livestockPx["Item"] = livestockPx["Item"].str.replace("Meat live weight, ","")
livestockPx["Item"] = livestockPx["Item"].str.capitalize()
livestockPx.drop(axis=0, columns={"Area Code", "Item Code"}, inplace = True)

meatPx = filterdf(df, 'Item', ['Meat, '])
meatPx = splitdf("Meat_Px_ton", "Producer Price (USD/tonne)", meatPx)
meatPx["Item"] = meatPx["Item"].str.replace("Meat, ","")
meatPx["Item"] = meatPx["Item"].str.capitalize()
meatPx.drop(axis=0, columns={"Area Code", "Item Code"}, inplace = True)

In [ ]:
# merge the DFs on country and year. This works the equivalent of a vlookup in excel
Milk_Livestock_Px_df = livestockPx.merge(meatPx, 
                                         how = 'left', 
                                         on =  ('Area', 'Item', 'Year'))

Milk_Livestock_Px_df = Milk_Livestock_Px_df.merge(dfDairyPx, 
                                                  how = 'left', 
                                                  on = ('Area', 'Item', 'Year'))

Milk_Livestock_Px_df = remove0orNAN(Milk_Livestock_Px_df, ['Year'])

displayInfo(Milk_Livestock_Px_df)

In [ ]:
print(Milk_Livestock_Px_df.Area.unique())
print(Milk_Livestock_Px_df.Item.unique())
print(Milk_Livestock_Px_df.Year.unique())

In [ ]:
ireland = Milk_Livestock_Px_df[Milk_Livestock_Px_df["Area"] == "Ireland"]

In [ ]:
ireland.tail(50)